In [1]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication

In [2]:
comm = UnityCommunication()

In [3]:
scene = 1
scene_graph = "TrimmedTestScene" + str(scene) + "_graph"
executable_program_path = "../dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/*/*.txt"
executable_program_list = []
for file_path in glob.glob(executable_program_path):
    executable_program_list.append(file_path.replace("../dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/", ""))

In [4]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=N83f14344cfdc478e99f9efd8fd441840 (<class 'rdflib.graph.Graph'>)>

In [5]:
def get_activity_from_ontology(activity_type):
    results = []
    qres = rdf_g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?activity where { 
    ?activity rdfs:subClassOf :%s .
 } 
       """ % activity_type)

    result = ""
    for row in qres:
        activity = "".join(row).replace("http://www.owl-ontologies.com/VirtualHome.owl#","")
        arr = activity.split("_")
        arr[0] = arr[0].capitalize()
        activity = " ".join(arr)
        result = activity
        results.append(result)
    return results

In [6]:
# set character
program_list = []
comm.reset(scene-1)
comm.add_character('chars/Female2')

True

In [7]:
# load environment
success, g = comm.environment_graph();

In [8]:
#nodes = g["nodes"]
#edges = g["edges"]
#max_id = nodes[len(nodes)-1]["id"]

In [9]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [10]:
data_path = "../dataset/programs_processed_precond_nograb_morepreconds/withoutconds/*/*.txt"
program_list = []
for file_path in glob.glob(data_path):
    file_name = file_path.replace("../dataset/programs_processed_precond_nograb_morepreconds/withoutconds/", "")
    if file_name in executable_program_list:
        program_name, description, list_of_steps = generate_list_of_steps(file_path)
        program_list.append({
            "file_name":file_name,
            "name": program_name,
            "description": description,
            "list_of_steps": list_of_steps
        })

In [11]:
def get_activity_program(category):
    #unexecutable = ["Take shower", "Take shoes off", "Wash teeth", "Wash face", "Dust", "Clean toilet", "Clean room", "Scrubbing living room tile floor is once week activity for me", "Clean mirror", "Play games", "Play on laptop", "Read on sofa"]
    unexecutable = []
    executable = []
    activities = get_activity_from_ontology(category)
    for activity_name in activities:
        if activity_name in unexecutable:
            continue
        results = [program for program in program_list if program["name"] == activity_name]
        if len(results) == 0:
            print("Nothing: " + activity_name)
        else:
            print("Success: " + activity_name)
            executable.append({"activity_name": activity_name, "results": results})
    return executable

In [12]:
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [13]:
def export(activity_name, graph_state_list, activity_cnt):
    os.mkdir("graph_state_list/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt)
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list/scene" + str(scene) + "/"  + activity_name + "/" + activity_cnt + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/activityList-program.txt", 'w') as f:
        for s in executed_program:
            f.write("%s\n" % s)

    with open("graph_state_list/scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/program-description.txt", 'w') as f:
        f.write("%s\n" % activity["name"])
        f.write("%s\n" % activity["description"])

In [14]:
executable_activity_list = []
for activity_class in ["BedTimeSleep", "EatingDrinking", "FoodPreparation", "GettingReady", "HouseArrangement", "HouseCleaning", "HygieneStyling", "Leisure", "Other", "PhysicalActivity", "SocialInteraction", "Work"]:
    executable_activity_list.extend(get_activity_program(activity_class))

Nothing: Wake me up
Nothing: Get ready for bed
Success: Sleep
Nothing: Put them to sleep
Nothing: Put alarm clock in bedroom
Success: Take nap
Nothing: Go to bed
Nothing: Read them bedtime story
Success: Go to sleep
Nothing: Bed time
Nothing: Get something to drink
Success: Eat dinner
Nothing: Eat cereal
Nothing: Eat meals
Nothing: Have dinner
Success: Eat
Nothing: Dining
Nothing: Eat snacks and drink tea
Nothing: Have ice coffee
Success: Drink
Nothing: Dance for table as we eat
Nothing: Eat while watching tv
Nothing: Eat dessert
Nothing: Eat family meals
Nothing: Eat cheese
Nothing: Eat ice cream
Nothing: Eat breaksfast
Nothing: Eat donuts
Success: Have snack
Nothing: Eat on fancy china
Nothing: Squeeze lemon
Success: Chop vegetables
Success: Make toast
Success: Toast bread
Success: Cook some food
Nothing: Make iced coffee
Success: Prepare pot of boiling water
Nothing: Make cereal
Success: Make coffee
Nothing: Cut steak
Nothing: Prepare sandwitch
Nothing: Bake
Nothing: Fix sandwhich
N

In [15]:
for executable_activity in executable_activity_list:
    activity_list = executable_activity["results"]
    activity_name = executable_activity["activity_name"]
    activity_cnt = 0
    try:
        os.mkdir("graph_state_list/scene" + str(scene) + "/" + activity_name)
        for activity in activity_list:
            try:
                comm.reset(scene-1)
                comm.add_character('chars/Female2')
                success, g = comm.environment_graph();
                print(activity["file_name"])
                script = activity["list_of_steps"]
                preconds = add_preconds.get_preconds_script(script).printCondsJSON()
                g = check_programs.translate_graph_dict_nofile(g)
                info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
                message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
                if message != "Script is executable":
                    print(message)
                    print("skipped")
                    continue
                # if delete randomly placced objects 
                graph_state_list = delete2000(graph_state_list)
                print(message)
                executed_program = []
                for x in executed_script:
                    executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

                export(activity_name, graph_state_list, str(activity_cnt))
                activity_cnt += 1
            except Exception as e:
                print(e.args)
    except Exception as e:
        print(e.args)

results_text_rebuttal_specialparsed_programs_upworknturk_second/split1_3.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split80_3.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split78_4.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split104_1.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split141_3.txt
Script is executable
results_intentions_march-13-18/file509_2.txt
("check_putoff() missing 1 required positional argument: 'char_index'",)
results_intentions_march-13-18/file1027_2.txt
Script is executable
results_intentions_march-13-18/file140_2.txt
Script is executable
results_intentions_march-13-18/file244_2.txt
Script is executable
results_intentions_march-13-18/file181_1.txt
Script is executable
results_intentions_march-13-18/file736_1.txt
Script is executable
results_intentions_march-13-18/file269_2.txt
Script is executable
resul

Script is executable
results_text_rebuttal_specialparsed_programs_upwork_july/split1_5.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split111_1.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_robot/split51_1.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split9_9.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split13_1.txt
("check_putoff() missing 1 required positional argument: 'char_index'",)
results_text_rebuttal_specialparsed_programs_upwork_july/split11_10.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split95_4.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split75_3.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split40_1.txt
Script is executable
results_text_rebuttal_specialpars

results_text_rebuttal_specialparsed_programs_upworknturk_second/split40_2.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_robot/split67_2.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_third/split74_2.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_robot/split76_3.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split22_2.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split65_2.txt
Script is executable
results_text_rebuttal_specialparsed_programs_upworknturk_second/split21_2.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_robot/split83_3.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_robot/split74_3.txt
Script is executable
results_text_rebuttal_special

results_text_rebuttal_specialparsed_programs_turk_third/split44_1.txt
("check_putoff() missing 1 required positional argument: 'char_index'",)
results_text_rebuttal_specialparsed_programs_turk_third/split68_3.txt
("check_putoff() missing 1 required positional argument: 'char_index'",)
results_text_rebuttal_specialparsed_programs_turk_third/split70_1.txt
("check_putoff() missing 1 required positional argument: 'char_index'",)
results_intentions_march-13-18/file263_1.txt
Script is executable
results_intentions_march-13-18/file737_1.txt
Script is executable
results_intentions_march-13-18/file385_2.txt
('cd_player',)
results_intentions_march-13-18/file199_1.txt
Script is executable
results_intentions_march-13-18/file363_2.txt
('dvd_player',)
results_intentions_march-13-18/file305_2.txt
Script is executable
results_intentions_march-13-18/file23_1.txt
Script is executable
results_intentions_march-13-18/file59_1.txt
('execute() takes 5 positional arguments but 6 were given',)
results_intentio

('execute() takes 5 positional arguments but 6 were given',)
results_intentions_march-13-18/file544_1.txt
Script is executable
results_intentions_march-13-18/file1073_1.txt
Script is executable
results_intentions_march-13-18/file15_2.txt
Script is executable
results_intentions_march-13-18/file208_2.txt
Script is executable
results_intentions_march-13-18/file897_2.txt
Script is executable
results_intentions_march-13-18/file1088_1.txt
Script is executable
results_intentions_march-13-18/file484_1.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_third/split3_4.txt
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_turk_third/split27_3.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split146_1.txt
Script is executable
results_text_rebuttal_specialparsed_programs_turk_third/split28_2.txt
('execute() takes 5 positional arguments but 6 were g

Script is executable
results_intentions_march-13-18/file708_1.txt
Script is executable
results_intentions_march-13-18/file737_2.txt
Script is executable
results_intentions_march-13-18/file345_1.txt
Script is executable
results_intentions_march-13-18/file511_1.txt
Script is executable
results_intentions_march-13-18/file1083_1.txt
('electrical_outlet',)
results_intentions_march-13-18/file205_1.txt
Script is executable
results_intentions_march-13-18/file120_1.txt
Script is executable
results_intentions_march-13-18/file616_1.txt
Script is executable
results_intentions_march-13-18/file627_1.txt
Script is executable
results_intentions_march-13-18/file585_1.txt
('floor_lamp',)
results_intentions_march-13-18/file443_2.txt
('floor_lamp',)
results_intentions_march-13-18/file705_1.txt
Script is executable
results_intentions_march-13-18/file835_2.txt
Script is executable
results_intentions_march-13-18/file780_1.txt
Script is executable
results_intentions_march-13-18/file445_1.txt
Script is executa

results_intentions_march-13-18/file381_1.txt
Script is executable
results_intentions_march-13-18/file451_1.txt
Script is executable
results_intentions_march-13-18/file429_1.txt
Script is executable
results_intentions_march-13-18/file287_1.txt
('execute() takes 5 positional arguments but 6 were given',)
results_intentions_march-13-18/file220_2.txt
Script is executable
results_intentions_march-13-18/file363_1.txt
Script is executable
results_intentions_march-13-18/file575_2.txt
Script is executable
results_intentions_march-13-18/file412_2.txt
Script is executable
results_intentions_march-13-18/file1063_2.txt
Script is executable
results_intentions_march-13-18/file612_1.txt
Script is executable
results_intentions_march-13-18/file279_2.txt
Script is executable
results_intentions_march-13-18/file203_2.txt
Script is executable
results_intentions_march-13-18/file38_1.txt
('execute() takes 5 positional arguments but 6 were given',)
results_intentions_march-13-18/file469_2.txt
('execute() takes